In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import torch

In [2]:
# dataset name
name = "AIC_2024"

In [3]:
if fo.dataset_exists(name):
    fo.delete_dataset(name)
    
dataset = fo.Dataset.from_images_dir(
    name=name, 
    images_dir="../data", 
    recursive=True
)

 100% |███████████| 106589/106589 [8.6s elapsed, 0s remaining, 12.2K samples/s]       


In [4]:
# if you want to automatically open the browser, just uncomment session.open_tab()
# or you can visit directly : http://localhost:5151/datasets/AIC_2024

session = fo.launch_app(dataset, auto=False)
# session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [5]:
print(dataset.first())

<Sample: {
    'id': '66d19484ffd52f2a41d7ca02',
    'media_type': 'image',
    'filepath': '/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/data/batch1/keyframes/keyframes_L01/L01_V001/001.jpg',
    'tags': [],
    'metadata': None,
}>


In [7]:
# run in about 35 seconds
for sample in dataset:
    _, sample['video'], sample['keyframe_id'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()